<a href="https://colab.research.google.com/github/abuelnasr0/Semantic-Search-project/blob/main/Notebooks/vector_database/populate_vdb_with_papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install grpcio==1.49.1 -q
!pip install pymilvus==2.2.11 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.4 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00


In [3]:
uri = ""
token = ""

In [4]:
from pymilvus import connections, Collection

connections.connect("default",
                  uri=uri,
                  token=token)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
PATH = '/content/drive/MyDrive/Graduation_project/Data/Papers/Arxiv_cleaned/new_data.json'

In [13]:
import json

with open(PATH, encoding='utf-8', mode='r') as json_file:
  loaded_papers = json.load(json_file)

In [14]:
# take only 50 000 of the papers
import random

MAX_NUM_PAPERS = 50_000

papers = loaded_papers
random.shuffle(papers)
papers = papers[0:MAX_NUM_PAPERS]

In [15]:
max_title_chars = 0
max_abstract_chars = 0

for paper in papers:
  max_abstract_chars = max(max_abstract_chars, len(paper['abstract']))
  max_title_chars = max(max_title_chars, len(paper['title']))

print("max title chars = ", max_title_chars)
print("max abstract chars = ", max_abstract_chars)

max title chars =  217
max abstract chars =  3658


In [16]:
max_id_chars = 0

for paper in papers:
  max_id_chars = max(max_id_chars, len(paper['id']))

print("max id chars = ", max_id_chars)

max id chars =  10


In [17]:
papers_ids = []
titles = []
abstracts = []

for paper in papers:
  papers_ids.append(paper['id'])
  titles.append(paper['title'])
  abstracts.append(paper['abstract'])

In [18]:
from sentence_transformers import SentenceTransformer
import torch
import os
model_name = "/content/drive/MyDrive/Graduation_project/Models/msmarco-bert-base-dot-v5-v2-Titles-wiht_150000_samples"

# model_name = os.getenv('MODEL_NAME', 'msmarco-bert-base-dot-v5')
assert model_name, "The model name is not avilable"

model = SentenceTransformer(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print('After Checking Cude: {}'.format(model.device))

DEBUG:jaxlib.mlir._mlir_libs:Initializing MLIR with module: _site_initialize_0
DEBUG:jaxlib.mlir._mlir_libs:Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/usr/local/lib/python3.10/dist-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
DEBUG:jax._src.path:etils.epath found. Using etils.epath for file I/O.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /content/drive/MyDrive/Graduation_project/Models/msmarco-bert-base-dot-v5-v2-Titles-wiht_150000_samples
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


After Checking Cude: cuda:0


In [19]:
embeddings = model.encode(abstracts, batch_size=256)

Batches:   0%|          | 0/196 [00:00<?, ?it/s]

In [22]:
embeddings = embeddings.tolist()
len(embeddings)

50000

In [31]:
NUM_ROWS = 10_000

data = [papers_ids[NUM_ROWS*4:NUM_ROWS*5], embeddings[NUM_ROWS*4:NUM_ROWS*5], titles[NUM_ROWS*4:NUM_ROWS*5], abstracts[NUM_ROWS*4:NUM_ROWS*5]]
papers_collection = Collection("research_papers")
papers_collection.load()
papers_collection.insert(data)
papers_collection.flush()
print(papers_collection.num_entities)

50000


In [44]:
# Search test
query = "machine learning"
query_embedding = model.encode(query)
query_E = query_embedding.tolist()
top_k = 15

search_params = {"metric_type": "IP"}

papers_collection = Collection("research_papers")
papers_collection.load()


results = papers_collection.search(
    data = [query_E],
    anns_field = "embedding",
    param = search_params,
    output_fields=['paper_id', 'title', 'abstract'],
    limit = top_k
)

# print(results[0].distances)


print(results)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['[\'id: 1612.04251, distance: 169.16384887695312, entity: {\\\'paper_id\\\': \\\'1612.04251\\\', \\\'title\\\': "TF.Learn: TensorFlow\\\'s High-level Module for Distributed Machine Learning", \\\'abstract\\\': "TF.Learn is a high-level Python module for distributed machine learning inside TensorFlow. It provides an easy-to-use Scikit-learn style interface to simplify the process of creating, configuring, training, evaluating, and experimenting a machine learning model. TF.Learn integrates a wide range of state-of-art machine learning algorithms built on top of TensorFlow\\\'s low level APIs for small to large-scale supervised and unsupervised problems. This module focuses on bringing machine learning to non-specialists using a general-purpose high-level language as well as researchers who want to implement, benchmark, and compare their new methods in a structured environment. Emphasis is put on ease of use, performance, documentation, and API consistency."}\', \'id: 1911.12593, distan

In [45]:
results = results[0]
ret = []

for match in results:
    data = {}
    data['id'] = match.id
    for key in match.entity.fields:
        data[key] = match.entity.get(key)
    ret.append(data)
for r in ret:
  print(r)

{'id': '1612.04251', 'paper_id': '1612.04251', 'title': "TF.Learn: TensorFlow's High-level Module for Distributed Machine Learning", 'abstract': "TF.Learn is a high-level Python module for distributed machine learning inside TensorFlow. It provides an easy-to-use Scikit-learn style interface to simplify the process of creating, configuring, training, evaluating, and experimenting a machine learning model. TF.Learn integrates a wide range of state-of-art machine learning algorithms built on top of TensorFlow's low level APIs for small to large-scale supervised and unsupervised problems. This module focuses on bringing machine learning to non-specialists using a general-purpose high-level language as well as researchers who want to implement, benchmark, and compare their new methods in a structured environment. Emphasis is put on ease of use, performance, documentation, and API consistency."}
{'id': '1911.12593', 'paper_id': '1911.12593', 'title': "Computer Systems Have 99 Problems, Let'